# Explore here

In [6]:
# Your code here
#libraris
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import chi2, SelectKBest


In [7]:
#Lectura de datos
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep = ";")
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41176 entries, 0 to 41175
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41176 non-null  object 
 3   education       41176 non-null  object 
 4   default         41176 non-null  object 
 5   housing         41176 non-null  object 
 6   loan            41176 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  object 
 9   day_of_week     41176 non-null  object 
 10  duration        41176 non-null  int64  
 11  campaign        41176 non-null  int64  
 12  pdays           41176 non-null  int64  
 13  previous        41176 non-null  int64  
 14  poutcome        41176 non-null  object 
 15  emp.var.rate    41176 non-null  float64
 16  cons.price.idx  41176 non-null  float64
 17  cons.conf.idx   41176 non-null 

In [8]:
#Filtrado de datos
data = data.drop_duplicates().reset_index(drop = True)
data.nunique()

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [10]:
#transformar los datos de objeto para podere tratarlos a lo largo del algorimo
# Crear el encoder
label_encoder = LabelEncoder()

# Aplicar el encoder a la columna
data['job_n'] = label_encoder.fit_transform(data['job'])
data['marital_n'] = label_encoder.fit_transform(data['marital'])
data['education_n'] = label_encoder.fit_transform(data['education'])
data['default_n'] = label_encoder.fit_transform(data['default'])
data['housing_n'] = label_encoder.fit_transform(data['housing'])
data['loan_n'] = label_encoder.fit_transform(data['loan'])
data['contact_n'] = label_encoder.fit_transform(data['contact'])
data['month_n'] = label_encoder.fit_transform(data['month'])
data['day_of_week_n'] = label_encoder.fit_transform(data['day_of_week'])
data['poutcome_n'] = label_encoder.fit_transform(data['poutcome'])
data['y_n'] = label_encoder.fit_transform(data['y'])

data.drop(columns=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome','y'], inplace = True)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41176 entries, 0 to 41175
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   duration        41176 non-null  int64  
 2   campaign        41176 non-null  int64  
 3   pdays           41176 non-null  int64  
 4   previous        41176 non-null  int64  
 5   emp.var.rate    41176 non-null  float64
 6   cons.price.idx  41176 non-null  float64
 7   cons.conf.idx   41176 non-null  float64
 8   euribor3m       41176 non-null  float64
 9   nr.employed     41176 non-null  float64
 10  job_n           41176 non-null  int64  
 11  marital_n       41176 non-null  int64  
 12  education_n     41176 non-null  int64  
 13  default_n       41176 non-null  int64  
 14  housing_n       41176 non-null  int64  
 15  loan_n          41176 non-null  int64  
 16  contact_n       41176 non-null  int64  
 17  month_n         41176 non-null 

In [12]:
#Dividiremos los datos en entrenamiento y test
X = data.drop("y_n", axis = 1)
y = data["y_n"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)


In [13]:
# Usamos xgb para conocer la importancia de las características
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42).fit(X_train, y_train)
data_imp = pd.DataFrame({'Feature':X_train.columns, 'Importance':xgb_model.feature_importances_*100})
data_imp = data_imp.sort_values(by='Importance', ascending=False)
data_imp


,Feature,Importance
9,nr.employed,67.783295
1,duration,4.361413
7,cons.conf.idx,3.215527
19,poutcome_n,2.674913
3,pdays,2.302261
6,cons.price.idx,1.952860
17,month_n,1.906862
16,contact_n,1.708631
8,euribor3m,1.553891
13,default_n,1.482968
